In [ ]:
!pip install -q s3fs

In [ ]:
# Define environment variables
os.environ["MINIO_KEY"] = "developer"
os.environ["MINIO_SECRET"] = "developer01"
os.environ["MINIO_ENDPOINT"] = "http://minio:9000"

 
"""
    Criando um Client de conexão com nosso Storage MinIO, atráves dele poderemos ler e gravar arquivos, somente como forma de testar a conexão.
    O objetivo principal é ler arquivos usando Spark e gravar novamente, após padronização, utilizando Spark no formato Delta.
    
"""

# Escrevendo arquivos de teste apenas para validar conexão
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': os.environ["MINIO_ENDPOINT"]}, # minio1 = minio container name
    key=os.environ["MINIO_KEY"],
    secret=os.environ["MINIO_SECRET"],
    use_ssl=False  # Set to True if MinIO is set up with SSL
)

teste = { "message": "hello world"}

with fs.open('bank-databr/landing/bacen/cartoes_trimestral/teste.json', 'w', encoding='utf-8') as f:
    json.dump(teste,f)

# Import and Libraries

import requests
import json
import os
import s3fs
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as F



spark = SparkSession.builder \
    .appName("country_data_analysis") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.endpoint", os.environ["MINIO_ENDPOINT"]) \
    .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_KEY"]) \
    .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET"]) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .enableHiveSupport() \
    .getOrCreate()

# PySpark Libraries
from pyspark.sql.functions import col, explode

df = spark.read \
          .option("inferSchema", True) \
          .json("s3a://bank-databr/landing/bacen/cartoes_trimestral/data_11_01_2025_22_02_56.json")

df = df.withColumn('value_struct', explode(col("value"))) \
       .select("value_struct.*")
     

df_exploded.show(n=5, truncate=True, vertical=True)